# A.C.E.L.S. Position Sensing NN Program

### Define Path to Model Files

In [ ]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)

MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

### Import Dependencies

In [ ]:
# Import libraries
import time
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense

### Import Dataset

In [ ]:
# Assign dataset to data variable
data = pd.read_csv('position_data_float_xyz_extended.csv')

### Pre-process data

In [ ]:
# Check dataset
data.head()
num_rows = data.shape[0]
print(num_rows)

In [ ]:
# Check datatype
data = data.sample(frac=1).reset_index(drop=True)
data.dtypes

# Convert dataframe to float 32
data32 = data.astype(np.float32)
data32.dtypes

In [ ]:
# Obtain data statistics
train_stats = data32.describe()
train_stats = train_stats.transpose()

print(train_stats)

# Separate Data into Feature and Target Variables
# The `_og` suffix refers to the original data without normalization
# It is assign to a variable to be later used for testing purposes
feature_data_og = data32[['s1','s2','s3','s4','s5','s6','s7','s8']]
target_data_og = data32[['x', 'y', 'z']]

# Check data shape and type
print(feature_data_og.shape[0])
print(type(feature_data_og.shape[0]))

# Split the data into  training and test sections
TRAIN_SPLIT = int(0.6 * feature_data_og.shape[0])
TEST_SPLIT = int(0.2 * feature_data_og.shape[0] + TRAIN_SPLIT)

feature_train_og, feature_test_og, feature_validate_og = np.split(feature_data_og, [TRAIN_SPLIT, TEST_SPLIT])
target_train_og, target_test_og, target_validate_og = np.split(target_data_og, [TRAIN_SPLIT, TEST_SPLIT])


In [ ]:
# Normalize data
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_data = norm(data32)

normed_data.head()

In [ ]:
# Separate Data into Feature and Target Variables
feature_data = normed_data[['s1','s2','s3','s4','s5','s6','s7','s8']]
target_data = normed_data[['x', 'y', 'z']]

In [ ]:
feature_data.head()

### Splitting the Data

In [ ]:
feature = feature_data
target = target_data

In [ ]:
# Assign 60% of data for training
# Assign 20% of data for testing
# Assign 20% pf data to validation
TRAIN_SPLIT = int(0.6 * feature.shape[0])
TEST_SPLIT = int(0.2 * feature.shape[0] + TRAIN_SPLIT)

feature_train, feature_test, feature_validate = np.split(feature, [TRAIN_SPLIT, TEST_SPLIT])
target_train, target_test, target_validate = np.split(target, [TRAIN_SPLIT, TEST_SPLIT])

# Check split data
feature_train.head()
target_train.head()

### Building and Training the Model

In [ ]:
# Create model with 8 input, 3 output and 5 hidden layers
model = tf.keras.Sequential()
model.add(Dense(60, activation='tanh', input_shape=(8,)))
model.add(Dense(80, activation='tanh'))
model.add(Dense(80, activation='tanh'))
model.add(Dense(60, activation='tanh'))
model.add(Dense(30, activation='tanh'))
model.add(Dense(3))
model.compile(optimizer='nadam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
# Train model
history_1 = model.fit(feature_train, target_train, epochs=50, batch_size=64, validation_data=(feature_validate, target_validate))
# Check Mean Absolute Error
test_loss, test_mae = model.evaluate(feature_test, target_test, verbose=0) 
print('Testing set Mean Abs Error: {:5.3f} mm'.format(test_mae))

In [ ]:
# Save model to disk
model.save(MODEL_TF)

## Plot Metrics and Analyse Model Accuracy

1. Mean Squared Error

In [ ]:
# Plot Mean Squared Error
train_loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Skip first 50 values and replot graph
SKIP = 50

plt.plot(epochs[SKIP:], train_loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

2. Mean Absolute Error

In [ ]:
plt.clf()
# Draw a graph of mean absolute error, which is another way of
# measuring the amount of error in the prediction.
train_mae = history_1.history['mae']
val_mae = history_1.history['val_mae']

plt.plot(epochs[SKIP:], train_mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and Validation Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
# Calculate and print the loss on our test dataset
test_loss, test_mae = model.evaluate(feature_test, target_test)

# Make predictions based on our test dataset
target_test_pred = model.predict(feature_test)

# print(feature_test)
# print(target_test)
#print(target_test_pred)

# print(feature_test)

# Model Evaluation

In [ ]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

df_sensors = feature_test
df_coordinates = target_test
coordinates2 = target_test_pred

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# scatter3D requires x, y, and z to be one-dimensional arrays
x = df_coordinates.iloc[:, 0]
y = df_coordinates.iloc[:, 1]
z = df_coordinates.iloc[:, 2]

x2 = coordinates2[:, 0]
y2 = coordinates2[:, 1]
z2 = coordinates2[:, 2]

ax.scatter3D(x2, y2, z2, c='red', s=8, alpha=0.5, label='Model predictions')  

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.scatter3D(x, y, z, c='blue', s=15, label='Actual Normalized Values')

plt.legend()  # Show legend to differentiate between the two sets
plt.show()

diff_list = []
diff_tracker = 0
for i, (actual, predicted) in enumerate(zip(x, x2)):
    diff = actual/predicted*100
    diff_tracker += diff
    
avrg_diff = diff_tracker/i

print(f"Average difference {avrg_diff}")


In [ ]:
# Check Mean Absolute Error
test_loss, test_mae = model.evaluate(feature_test, target_test, verbose=0)

print('Testing set Mean Abs Error: {:5.3f} mm'.format(test_mae))

# Convert to numpy arrays if they are pandas DataFrames
original_data = target_test.to_numpy()
predicted_data = target_test_pred

# Calculate Mean Squared Error
mse = np.mean((original_data - predicted_data) ** 2)

# Convert to Root Mean Squared Error
rmse = np.sqrt(mse)

# Estimate the range of the data
data_range = np.max(original_data) - np.min(original_data)

# Calculate accuracy percentage
accuracy = (1 - rmse / data_range) * 100
accuracy_percentage = np.clip(accuracy, 0, 100)  # Ensure the percentage is between 0 and 100

print("\n#-----------------------------------------------------")
print(f"# Model Accuracy: {accuracy_percentage:.2f}%")
print("#-----------------------------------------------------")


## Denormalize output 

In [ ]:
# Check model output values
pred_df = pd.DataFrame(target_test_pred, columns = ['x','y','z'])
pred_df.head()

In [ ]:
# Denormalize Values
def denorm(x):
  return (x * train_stats['std']) + train_stats['mean']

denormed_data = denorm(pred_df)
denormed_feature = denorm(feature)
denormed_target = denorm(target)

print(denormed_data.shape)
print(denormed_feature.shape)

print(f"Denormalized Mean Absolute Error {denorm(test_mae)}")

# Denormalized Evaluation

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Assuming df_sensors is the 808x8 DataFrame, df_coordinates is the 808x3 DataFrame,
# and coordinates2 is the second 808x3 NumPy array
df_sensors = denormed_feature[['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']]
df_coordinates = denormed_target[['x', 'y', 'z']]
coordinates2 = denormed_data[['x', 'y', 'z']]

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# scatter3D requires x, y, and z to be one-dimensional arrays
x2 = coordinates2.iloc[:, 0]
y2 = coordinates2.iloc[:, 1]
z2 = coordinates2.iloc[:, 2]

ax.scatter3D(x2, y2, z2, c='red', s=8, label='Model predictions')  # Plot second set with different color

x = df_coordinates.iloc[:, 0]
y = df_coordinates.iloc[:, 1]
z = df_coordinates.iloc[:, 2]

ax.scatter3D(x, y, z, c='blue', s=15, label='Actual values')
ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')
ax.set_zlabel('')
ax.text2D(1.02, 0.55, 'Z (mm)', transform=ax.transAxes, verticalalignment='center')

plt.legend()

plt.savefig("scatter_plot_4.svg", format='svg', bbox_inches='tight', pad_inches=0)
 
plt.show()



In [ ]:
print(target_test_pred.shape)

print(target_test.shape)

print(denormed_target.shape)


### Model Accuracy

In [ ]:
# Convert to numpy arrays if they are pandas DataFrames
original_data = target_test_og.to_numpy()
predicted_data = coordinates2

# Calculate Mean Squared Error
mse = np.mean((original_data - predicted_data) ** 2)

# Convert to Root Mean Squared Error
rmse = np.sqrt(mse)

# Estimate the range of the data
data_range = np.max(original_data) - np.min(original_data)

# Calculate accuracy percentage
accuracy = (1 - rmse / data_range) * 100
accuracy_percentage = np.clip(accuracy, 0, 100)  # Ensure the percentage is between 0 and 100

# accuracy_percentage

print("\n#-----------------------------------------------------")
print(f"# Denormalized Model Accuracy: {accuracy_percentage:.2f}%")
print("#-----------------------------------------------------")


# Generate aTensorflow Lite Model

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for _ in range(500):
    yield([feature_train.astype(np.float32)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(MODEL_NO_QUANT_TFLITE)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define your input here. I am using random for the simplicity
input_data = feature_test.to_numpy().astype(np.float32)

print(input_data)
print(input_data.shape[0])

start_time = time.time()

# Convert DataFrame to numpy array for correct indexing

# Assuming `feature_test_np` is a 2D numpy array with shape (808, 8)
for i in range(input_data.shape[0]):
    single_instance = np.expand_dims(input_data[i], axis=0)  # Reshape from (8,) to (1, 8)

    interpreter.set_tensor(input_details[0]['index'], single_instance)
    interpreter.invoke()

end_time = time.time()

# Time taken
print(f"Time taken for non-quantized prediction: {end_time - start_time} seconds")


In [ ]:

import time
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(MODEL_TFLITE)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Define your input here. I am using random for the simplicity
# Assuming `input_data_normalized` is your normalized test data
input_data_quantized = (feature_test * 127).astype(np.int8)
input_data_quantized_np = input_data_quantized.to_numpy()
print(input_data_quantized)
print(input_data_quantized_np.shape[0])

start_time = time.time()
# Assuming `input_data_quantized_np` is a 2D numpy array with shape (808, 8)
for i in range(input_data_quantized_np.shape[0]):
    single_instance = np.expand_dims(input_data_quantized_np[i], axis=0)  # Reshape from (8,) to (1, 8)

    interpreter.set_tensor(input_details[0]['index'], single_instance)
    interpreter.invoke()


end_time = time.time()

# Time taken
print(f"Time taken for quantized prediction: {end_time - start_time} seconds")

In [ ]:
size_tf_small = os.path.getsize(MODEL_NO_QUANT_TFLITE)
pd.DataFrame.from_records([["Tensorflow", f"{size_tf_small} bytes",""]], columns = ["Model", "Size", ""], index="Model")

### Compare Quantized and Non Quantized Model

In [ ]:
# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)

In [ ]:
# Compare sizes
pd.DataFrame.from_records(
    [
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes "],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes"]],
     columns = ["Model", "Size"], index="Model")

### Generate TF Lite for Microcontroller Model

In [ ]:
# Install xxd if it is not available
!apt-get update && apt-get -qq install xxd
# Convert to a C source file, i.e, a TensorFlow Lite for Microcontrollers model
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/position_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}